## Flowers Dataset 실습

1. 데이터셋
    - [링크](https://www.kaggle.com/datasets/aksha05/flower-image-dataset)를 참고하여 아래 코드를 구성하였습니다.
    - 데이터셋은 총 733개 이미지를 갖고 있습니다.
    - 학습셋 586개 이미지, 테스트셋 147개 이미지입니다.

2. 실습내용
    - 아래 예시코드를 참고하여 이미지 분류 성능을 높이는 실습입니다.
    - 아래 예시는 간단하게 폴더에서 이미지 파일을 읽어와서 숫자로 변경하고, 해당 숫자를 이용하여 SVM 기본 모델을 학습했습니다. 정확도는 약 14%로 매우 저조합니다.
    - 아래 내용을 참고하여 다양한 시도(모델 변경, 이미지 features 전처리 등)를 하여 모델 성능을 높여보세요!

3. 참고
    - 해당 실습은 수강생분들의 현재까지 머신러닝에 대한 이해를 돕기위한 실습입니다.
    - 데이터셋이나 코드 등에 문제가 있을경우 말씀 부탁드리겠습니다! 

In [1]:
# !pip install --upgrade pip
# !pip install tqdm

In [2]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from PIL import Image

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC

### Load dataset

In [3]:
list_of_files = []
for filename in os.listdir('flowers'):
    flower_name = filename.split('_')[0]
    filepath = os.path.join('flowers', filename)
    
    list_of_files.append((flower_name, filepath))

In [4]:
print(len(list_of_files))

733


In [5]:
df = pd.DataFrame(data=list_of_files, columns=['label', 'filepath'])
print(df.head())

           label                         filepath
0  bougainvillea  flowers\bougainvillea_00002.jpg
1  bougainvillea  flowers\bougainvillea_00003.jpg
2  bougainvillea  flowers\bougainvillea_00004.jpg
3  bougainvillea  flowers\bougainvillea_00005.jpg
4  bougainvillea  flowers\bougainvillea_00006.jpg


In [6]:
train_x, test_x, original_train_y, original_test_y = train_test_split(df['filepath'].values, df['label'].values, 
                                                                      test_size=0.2)

In [7]:
def convert_img_to_arr(filepath):
    img = np.array(Image.open(filepath))
    resize_img = np.resize(img, (10, 10, 3) )
    resize_img = resize_img.astype('float32')
    resize_img /= 255
    
    return resize_img

In [8]:
train_x = [convert_img_to_arr(x).reshape(-1, 10 * 10 * 3) for x in tqdm(train_x)]
test_x = [convert_img_to_arr(x).reshape(-1, 10 * 10 * 3) for x in tqdm(test_x)]

100%|██████████| 147/147 [00:06<00:00, 21.43it/s]


In [9]:
train_x = [item for sublist in train_x for item in sublist]
test_x = [item for sublist in test_x for item in sublist]

In [10]:
labelencoder = LabelEncoder()
train_y = labelencoder.fit_transform(original_train_y)
test_y = labelencoder.fit_transform(original_test_y)

In [11]:
len(train_x), len(test_x)

(586, 147)

In [12]:
len(train_x[0]), len(test_x[0])

(300, 300)

### Train model and Evaluate

In [13]:
model = SVC(kernel='rbf', verbose=True)
model.fit(train_x, train_y)

[LibSVM]

SVC(verbose=True)

In [14]:
pred = model.predict(test_x)

In [15]:
accuracy_score(test_y, pred)

0.22448979591836735

In [16]:
confusion_matrix(test_y, pred)

array([[9, 0, 3, 3, 0, 1, 2, 0, 1, 0],
       [1, 1, 1, 4, 0, 0, 6, 0, 2, 0],
       [1, 1, 4, 3, 0, 1, 3, 0, 1, 0],
       [0, 3, 1, 3, 1, 1, 3, 0, 0, 2],
       [0, 1, 0, 6, 2, 1, 2, 0, 2, 1],
       [0, 0, 1, 3, 1, 2, 2, 0, 2, 0],
       [0, 1, 1, 6, 0, 1, 6, 0, 2, 2],
       [2, 0, 3, 2, 0, 0, 2, 0, 1, 1],
       [1, 4, 0, 6, 0, 0, 3, 0, 3, 1],
       [1, 0, 0, 2, 0, 1, 2, 0, 2, 3]], dtype=int64)